In [3]:
%%capture
from data_loader import DataLoader
from data_preprocessor import DataPreprocessor
from text_cleaner import TextCleaner
from feature_extractor import FeatureExtractor
from stress_model import StressModel
#from svm_model import StressModel
from stress_predictor import StressPredictor
import pandas as pd
import numpy as np


In [ ]:
# 22. Path to the CSV file
csv_path = "Stress.csv"

In [ ]:
# 3. Load the data using DataLoader
data_loader = DataLoader(csv_path)
df = data_loader.load_data()

In [ ]:
# 4. Prepare the data using TextCleaner and DataPreprocessor
cleaner = TextCleaner()
preprocessor = DataPreprocessor(cleaner=cleaner)
X_train, X_test, y_train, y_test = preprocessor.process(df)

In [18]:
X_test.shape

(374,)

In [ ]:
pd.set_option('display.max_columns', None)  

In [ ]:
# 5. Feature extraction
extractor = FeatureExtractor()
X_train_vec = extractor.fit_transform(X_train)
X_test_vec = extractor.transform(X_test)


In [ ]:
# 6. Model training and evaluation
model = StressModel()
model.train(X_train_vec, y_train)
report_df = model.evaluate(X_test_vec, y_test)

In [10]:
report_df

,precision,recall,f1-score,support
0,0.755814,0.792683,0.773810,164.000000
1,0.831683,0.800000,0.815534,210.000000
accuracy,0.796791,0.796791,0.796791,0.796791
macro avg,0.793749,0.796341,0.794672,374.000000
weighted avg,0.798414,0.796791,0.797238,374.000000


In [ ]:
# 7. Make a prediction from user input text
predictor = StressPredictor(model, extractor, cleaner)
example_text = " I've been using drugs but I'm happy and fine"
result = predictor.predict_from_text(example_text)
print(f"Tahmin: {result}")

Tahmin: Stresli değil (%46.54 stres olasılığı)


In [13]:
predictor.predict_from_text("I'm feeling pretty good today!")

'Stresli (%59.14 stres olasılığı)'

In [13]:
predictor.predict_from_text("I'm fine. Just tired of everything.")

'Stresli (%74.87 stres olasılığı)'

In [11]:
predictor.predict_from_text("I keep telling everyone I'm okay, but I don't even know anymore.")

'Stresli (%87.19 stres olasılığı)'

In [17]:
predictor.predict_from_text("Some days are better than others, today just isn’t one of them.")

'Stresli değil (%46.71 stres olasılığı)'

In [ ]:
# Make predictions on the test set and compare the results.
y_pred = model.predict(X_test_vec)

df_results = pd.DataFrame({
    "text": X_test,
    "actual": y_test.values,
    "predicted": y_pred
})

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)

In [ ]:
# Display the first few prediction results
print("İlk 5 tahmin sonucu:")
print(df_results.head())

İlk 5 tahmin sonucu:
                                                   text  actual  predicted
2592  exchange participation may optionally choose e...       0          0
2564  getting changed pair short standing lifted one...       1          1
1421  wan na break situation know guy cant help need...       0          1
435   couldnt thanksgiving money struggle family beg...       1          1
653   force move forward ridiculous still ptsd feel ...       1          1


In [ ]:
# Analyze the misclassified predictions
incorrect = df_results[df_results["actual"] != df_results["predicted"]]
print(f"\nToplam hatalı tahmin sayısı: {len(incorrect)}")
print("\nİlk 10 hatalı tahmin:")
print(incorrect.head(10))


Toplam hatalı tahmin sayısı: 76

İlk 10 hatalı tahmin:
                                                   text  actual  predicted
1421  wan na break situation know guy cant help need...       0          1
504   need make decision whole course life depending...       1          0
320   tend like driving long distance car people unf...       0          1
2112  typically involves simply leaving building ins...       0          1
2061  anyway long story short came blast dinner watc...       0          1
755   eventually reported text kept coming recieved ...       1          0
1939  nothing major underage drinking little pot was...       0          1
1462  cant beat em join em right ive learned okay an...       0          1
1832  three consecutive appointment specialist shift...       1          0
904   quit facebook twitter instragram essentially s...       0          1
